<a href="https://www.kaggle.com/code/romankovalchuk/lora-ua-gec?scriptVersionId=167968298" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!git clone https://github.com/Reennon/ua-gec-lora.git
!cd ua-gec-lora && pip install -r requirements.txt
!pwd && ls -a
# Install additional libs
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/trl.git@7630f877f91c556d9e5a3baa4b6e2894d90ff84c
!pip install ua_gec
!pip install datasets==2.16.0
# CD into the project directory
%cd ua-gec-lora
!git pull origin "feature/fine-tuning-research"
!git status

Cloning into 'ua-gec-lora'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 82 (delta 17), reused 71 (delta 12), pack-reused 0
Unpacking objects: 100% (82/82), 84.10 KiB | 3.50 MiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 1.6 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 13.3 MB/s 

In [2]:
import torch
from transformers import AutoModelForCausalLM, pipeline, Conversation, AutoTokenizer
from src.packages.utils.parameter_server import ParameterServer
import torch

2024-03-20 10:40:03.554298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 10:40:03.554415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 10:40:03.707069: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
print(torch.cuda.is_available())

True


In [4]:
device = "cuda"

In [5]:
parameter_server = ParameterServer()

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2" #"mistralai/Mistral-7B-Instruct-v0.2" # "bardsai/jaskier-7b-dpo-v6.1" # "microsoft/phi-2" # "bardsai/jaskier-7b-dpo-v6.1"

In [6]:
model = AutoModelForCausalLM.from_pretrained(base_model_name, load_in_4bit=True, device_map={'':torch.cuda.current_device()},)
model

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name, 
    use_fast=False, 
    padding_side="left",
)
tokenizer

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

LlamaTokenizer(name_or_path='mistralai/Mistral-7B-Instruct-v0.2', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [94]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,
    lora_alpha=32,
    bias="none",
    lora_dropout=0.05,  # Conventional
#     target_modules=[
#         "gate_proj", 
#         "down_proj", 
#         "up_proj", 
#         "q_proj", 
#         "v_proj", 
#         "k_proj", 
#         "o_proj",
#     ],
)

model.enable_input_require_grads()
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 10,485,760 || all params: 7,252,217,856 || trainable%: 0.14458694165295632


In [9]:
from langchain.prompts import PromptTemplate

template = """[INST]Given ("ORIGINAL_TEXT") text with errors, correct them if present, fulfilling GEC (Grammar Error Correction) Task for Ukrainian Language.
Consider following set of possible error types ("ERROR_TYPES"):
{error_types}
Throughout the text, if you would detect error ("ERROR"), fix it according to the structure:
("ERROR") => ("CORRECTION")
Where ("CORRECTION") Specifies how the error should be corrected, without reasoning the change.
The generated text ("FIXED_TEXT") should not contain original errors, additional generated text, comments, nor part of these instructions.
Correct only Ukrainian language.
Do not translate the text; perform grammar correction only for Ukrainian language.
Keep original information intact, prioritizing the semantics of original text being intact.

ORIGINAL_TEXT: {query}
FIXED_TEXT:
[/INST]"""

it_prompt = PromptTemplate(
    template=template,
    input_variables=['query', 'error_types']
)

In [10]:
!pip install nltk

In [11]:
import nltk
nltk.download('punkt')  # Download the necessary resources for sentence tokenization

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from src.packages.constants.error_constants import ErrorConstants
from src.packages.prompts.instruction_tuning_gec_prompts import InstructionTuningGecPrompts
from ua_gec import Corpus
corpus = Corpus(partition="train", annotation_layer="gec-only")
for doc in corpus:
    print("\n---Source starts:---\n")
    print(doc.source[:1000])         # "I likes it."
    print("\n---Source ends:---\n")
    print("\n---Target starts:---\n")
    print(doc.target[:1100])         # "I like it."
    print("\n---Target ends---\n")
    print("\n---Annotation starts:---\n")
    print(str(doc.annotated)[:1200])      # <AnnotatedText("I {likes=>like} it.")
    print(doc.meta.region)    # "Київська"
    print("\n---Annotation ends")
    print("\n---Prompt starts")
    prompt = it_prompt.format_prompt(
        query="".join(sent_tokenize(doc.source)[:6]),
        error_types=ErrorConstants.ERROR_TYPES
    )
    print(prompt)
    break


---Source starts:---

Byte for France або “Мій досвід ведення блогу у Instagram”
Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.

Сьогодні розповім про те як і навіщо мене занесло у Instagram. Якщо цікаво подивитися відразу на результат, то щиро прошу за цим посиланням – https://www.instagram.com/yevhenii_kanivets/

Моє бачення Instagram
Колись давно я прочитав статтю, чи просто коментарій – вже не згадаю. Але йшлося там про те, що найпопулярнішою соціальною мережею стане платформа на котрій можно буде лише ділитися світлинами та ставити лайки.

Було це за декілька років до появи усім відомого сервісу. Як же автору вдалося передбачити майбутнє? Дуже просто!

Instagram втілює глибинні бажання кожної людини:

емоційне збудження від перегляду гарних фото (читання – це менш природний процес, ніж споглядання)
нескінченне поглинан

In [13]:
InstructionTuningGecPrompts.PROMPT

PromptTemplate(input_variables=['error_types', 'query'], template='Given ("SOURCE") text, correct errors if present, fulfilling GEC (Grammar Error Correction) Task for Ukrainian Language.\nUse following set of errors ("ERROR_TYPES"):\n{error_types}\nIf you would detect error ("ERROR"), generate correction ("CORRECTION") following the structure and specifying error type ("ERROR_TYPE"):\n{{("ERROR")=>("CORRECTION"):::("ERROR_TYPE")}}\nOtherwise keep original text.\n\nSOURCE: {query}\nERROR_TYPES: {error_types}\nFINAL ANSWER:')

In [14]:
!git fetch && git pull

Already up to date.


In [15]:
prompt_len = len(tokenizer.tokenize(prompt.to_string()))
tokenizer_max_len = 768
max_correction_addtional_tokens = 0.1
max_new_tokens = prompt_len + int(prompt_len * 0.1)

print(f"""
Tokenizer max tokens: {tokenizer_max_len}
Prompt len: {prompt_len}
Max token difference because of corrections: {max_correction_addtional_tokens}
Max new tokens len (output without input): {max_new_tokens}
""")


Tokenizer max tokens: 768
Prompt len: 554
Max token difference because of corrections: 0.1
Max new tokens len (output without input): 609



In [16]:
# Fix padding token for Mistral and Phi-2 models
tokenizer.pad_token = "[PAD]"

In [17]:
model_inputs = tokenizer(
    prompt.to_string(), 
    max_length=tokenizer_max_len, 
    padding="max_length", 
    truncation=True, 
    return_tensors="pt"
)
model_inputs

{'input_ids': tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [18]:
response = peft_model.generate(
    input_ids=model_inputs["input_ids"].to(device),
    attention_mask=model_inputs["attention_mask"].to(device),
    max_new_tokens=max_new_tokens
)
response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


tensor([[    0,     0,     0,  ..., 22561,  2974,     2]], device='cuda:0')

In [19]:
decoded_outputs = tokenizer.batch_decode(response.detach().cpu().numpy(), skip_special_tokens=True)
text = decoded_outputs[0][len(prompt.to_string()):]
text

' Byte for France або "Мій досвід ведення блогу на Instagram"\n\nОстанні три місяці мого життя видалися надто засипаними подіями та емоціями, але нарешті у мене з\'явилося декілька вільних годин та малої енергії, щоб продовжити серію записів щодо мого досвіду блогерства.Сьогодні я розповім про те, як і для чого мене занесло на Instagram.Якщо цікаво подивитися відразу на результат, то щиро прошу за цим посиланням – <https://www.instagram.com/yevhenii_kanivets/>\n\nМоє бачення Instagram\n\nКолись давно я прочитав статтю, чи лише коментарій – вже не згадую.Але йшлося там про те, що найпопулярнішою соціальною мережею стане платформа, на якій можна буде лише розміщувати зображення та ставити лайки.Було це за кілька років до появи усім відомого сервісу.Як автору вдалося передбачити майбутнє?\n\n(Note: The text provided is grammatically correct in Ukrainian language. Therefore, there is no need for any correction.)'

In [20]:
print(decoded_outputs[0])

[INST]Given ("ORIGINAL_TEXT") text with errors, correct them if present, fulfilling GEC (Grammar Error Correction) Task for Ukrainian Language.
Consider following set of possible error types ("ERROR_TYPES"):
['Fluency', 'Grammar', 'Punctuation', 'Spelling']
Throughout the text, if you would detect error ("ERROR"), fix it according to the structure:
("ERROR") => ("CORRECTION")
Where ("CORRECTION") Specifies how the error should be corrected, without reasoning the change.
The generated text ("FIXED_TEXT") should not contain original errors, additional generated text, comments, nor part of these instructions.
Correct only Ukrainian language.
Do not translate the text; perform grammar correction only for Ukrainian language.
Keep original information intact, prioritizing the semantics of original text being intact.

ORIGINAL_TEXT: Byte for France або “Мій досвід ведення блогу у Instagram”
Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з

In [21]:
from difflib import SequenceMatcher
import re

def normalize_spaces(text):
    return ' '.join(text.split())

def highlight_changes(text1, text2):
    # Tokenize the texts into words
    words1 = re.findall(r'\w+|[^\w\s]', text1)
    words2 = re.findall(r'\w+|[^\w\s]', text2)


    # Find the unique words present in both texts
    all_words = set(words1 + words2)

    # Initialize a SequenceMatcher object
    matcher = SequenceMatcher(None, words1, words2)

    # Get the differences
    diff = matcher.get_opcodes()

    highlighted_text = []

    for op, start1, end1, start2, end2 in diff:
        if op == 'equal':
            # No change, just append the words as is
            highlighted_text.extend(words1[start1:end1])
        elif op == 'delete':
            # Word(s) removed, highlight with red
            for word in words1[start1:end1]:
                word = '\u0336'.join(word) + '\u0336'
                highlighted_text.append('\033[91m\033[1m' + word + '\033[0m')
        elif op == 'insert':
            # Word(s) added, highlight with green
            for word in words2[start2:end2]:
                highlighted_text.append('\033[92m\033[1m' + word + '\033[0m')
        elif op == 'replace':
            # Word(s) replaced, highlight with yellow
            for word in words2[start2:end2]:
                highlighted_text.append('\033[93m\033[1m' + word + '\033[0m')

    return ' '.join(highlighted_text)

def generate_original_corrected_texts(original_text, corrected_text, highlighted_comparison):
    # Split the original and corrected texts
    original_words = original_text.split()
    corrected_words = corrected_text.split()

    # Initialize empty lists for marked original and corrected texts
    marked_original_text = []
    marked_corrected_text = []

    # Track words from the original text that were removed
    removed_words = set(original_words) - set(corrected_words)

    # Track words from the corrected text that were added
    added_words = set(corrected_words) - set(original_words)

    # Mark removed words in the original text as red
    for word in original_words:
        if word in removed_words:
            marked_original_text.append('\033[91m\033[1m' + word + '\033[0m')
        else:
            marked_original_text.append(word)

    # Mark added words in the corrected text as green
    for word in corrected_words:
        if word in added_words:
            marked_corrected_text.append('\033[92m\033[1m' + word + '\033[0m')
        else:
            marked_corrected_text.append(word)

    return (' '.join(marked_original_text), ' '.join(marked_corrected_text), highlighted_comparison)

text1 = normalize_spaces("".join(sent_tokenize(doc.source)[:6]))
text2 = normalize_spaces(text[1:])

highlighted_text = highlight_changes(text1, text2)

original_text, corrected_text, _ = generate_original_corrected_texts(
    original_text=text1, 
    corrected_text=text2, 
    highlighted_comparison=highlighted_text)

print("Original Text:")
print(original_text)
print()

print("Corrected Text:")
print(corrected_text)
print()

print("Changes comparison:")
print(highlighted_text)


Original Text:
Byte for France або “Мій досвід ведення блогу у Instagram” Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.Сьогодні розповім про те як і навіщо мене занесло у Instagram.Якщо цікаво подивитися відразу на результат, то щиро прошу за цим посиланням – https://www.instagram.com/yevhenii_kanivets/ Моє бачення Instagram Колись давно я прочитав статтю, чи просто коментарій – вже не згадаю.Але йшлося там про те, що найпопулярнішою соціальною мережею стане платформа на котрій можно буде лише ділитися світлинами та ставити лайки.Було це за декілька років до появи усім відомого сервісу.Як же автору вдалося передбачити майбутнє?

Corrected Text:
Byte for France або "Мій досвід ведення блогу на Instagram" Останні три місяці мого життя видалися надто засипаними подіями та емоціями, але нарешті у мене з'явилося декілька вільних

In [104]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from nltk.tokenize import sent_tokenize

class UAGECDataset(Dataset):
    def __init__(
        self, 
        generator: object, 
        device: str,
        prompt: object,
        max_sentences=6,
        samples: int = None # if none will use all
    ):
        self.text_data = list(generator)
        
        if samples:
            self.text_data = self.text_data[:samples]
        
        self.max_sentences = max_sentences
        self.device = device
        self.prompt = prompt

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        sample = self.text_data[idx]
        
        inputs: str = self._preprocess_text(
            text=sample.source, 
            target_text=sample.target
        )
        encodings = self._tokenize_text(
            text=inputs,
        ).to(self.device)

        return {
            'prompt': inputs,
            'input_ids': encodings["input_ids"].squeeze(0),
            'attention_mask': encodings["attention_mask"].squeeze(0),
        }
    
    def _preprocess_text(self, text: str, target_text: str) -> torch.tensor:
        # Select top n sentences
        text = "".join(sent_tokenize(text)[:self.max_sentences])
        target_text = "".join(sent_tokenize(target_text)[:self.max_sentences])
        # Add instructions (prepend prompt)
        text = self._format_prompt(text=text)

        text = self._normalize_spaces(text=text)
        target_text = self._normalize_spaces(text=target_text)
        # Add target response to input text
        text += target_text
        
        return text
    
    def _format_prompt(self, text: str) -> str:
        return self.prompt.format_prompt(
            query=text,
            error_types=ErrorConstants.ERROR_TYPES
        ).to_string()
    
    def _tokenize_text(self, text: str):
        return tokenizer(
            text, 
            max_length=tokenizer_max_len, 
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )
    
    def _add_target(self, text: str, target_text: str):
        return self.tokenizer(
            text, 
            max_length=self.tokenizer_max_len, 
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )
    
    def _normalize_spaces(self, text):
        return ' '.join(text.split())

In [107]:
from ua_gec import Corpus

train_corpus = Corpus(partition="train", annotation_layer="gec-only")
test_corpus = Corpus(partition="test", annotation_layer="gec-only")
train_dataset, val_dataset = [UAGECDataset(
    generator=corpus,
    device=device,
    prompt=it_prompt,
    max_sentences=6,
    samples=samples
) for corpus, samples in [(train_corpus, 500), (test_corpus, 50)]]

In [108]:
train_dataset[0]

{'prompt': '[INST]Given ("ORIGINAL_TEXT") text with errors, correct them if present, fulfilling GEC (Grammar Error Correction) Task for Ukrainian Language. Consider following set of possible error types ("ERROR_TYPES"): [\'Fluency\', \'Grammar\', \'Punctuation\', \'Spelling\'] Throughout the text, if you would detect error ("ERROR"), fix it according to the structure: ("ERROR") => ("CORRECTION") Where ("CORRECTION") Specifies how the error should be corrected, without reasoning the change. The generated text ("FIXED_TEXT") should not contain original errors, additional generated text, comments, nor part of these instructions. Correct only Ukrainian language. Do not translate the text; perform grammar correction only for Ukrainian language. Keep original information intact, prioritizing the semantics of original text being intact. ORIGINAL_TEXT: Byte for France або “Мій досвід ведення блогу у Instagram” Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але о

In [109]:
from huggingface_hub import notebook_login

notebook_login()

In [125]:
from trl import SFTTrainer
from transformers import TrainingArguments

fine_tune_model_id = "rkovalchuk/mistral-7b-it-ua-gec"

# # Since the model is loaded in 4bit precision, use right-side padding for tokenizer
# tokenizer.padding_side = 'right'

training_arguments = TrainingArguments(
    output_dir=fine_tune_model_id.split("/")[-1],
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=2e-4,
    logging_steps=100,
    num_train_epochs=5,
    save_total_limit = 2,
    save_strategy="no",
    load_best_model_at_end=True,
    hub_private_repo=False,
    report_to=None,
    optim="paged_adamw_32bit",
)
peft_model = peft_model.to(device)
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="prompt",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=max_new_tokens,
    packing=False,

)

In [111]:
training_arguments.device

device(type='cuda', index=0)

In [126]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

60

In [127]:
import torch
import time
import gc
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

def clear_gpu_memory():
    torch.cuda.empty_cache()
    gc.collect()

def wait_until_enough_gpu_memory(min_memory_available, max_retries=10, sleep_time=5):
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(torch.cuda.current_device())

    for _ in range(max_retries):
        info = nvmlDeviceGetMemoryInfo(handle)
        if info.free >= min_memory_available:
            break
        print(f"Waiting for {min_memory_available} bytes of free GPU memory. Retrying in {sleep_time} seconds...")
        time.sleep(sleep_time)
    else:
        raise RuntimeError(f"Failed to acquire {min_memory_available} bytes of free GPU memory after {max_retries} retries.")

# Usage example
min_memory_available = 2 * 1024 * 1024 * 1024  # 2GB
clear_gpu_memory()
wait_until_enough_gpu_memory(min_memory_available)

In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
trainer.push_to_hub()